# Εργασία Ανάκτησης Πληροφορίας
## Δημιουργία μηχανής αναζήτησης 
### Βήμα 1. Συλλογή δεδομένων:

In [12]:
import json

# Διαβάζουμε το αρχείο JSON
with open('articles.json', 'r', encoding='utf-8') as file:
    articles = json.load(file)

# Εξάγουμε τα δεδομένα των άρθρων
documents = []
for article in articles:
    doc = {
        'ID': article.get('ID'),
        'Title': article.get('Title'),
        'Body': article.get('Body', ''),
        'URL': article.get('URL'),
    }
    documents.append(doc)

# Εμφανίζουμε τον αριθμό των εγγράφων που συλλέχθηκαν
print(f"Συλλέχθηκαν {len(documents)} άρθρα.")

# Παράδειγμα εμφάνισης ενός εγγράφου
print(json.dumps(documents[0], indent=4, ensure_ascii=False))


Συλλέχθηκαν 1000 άρθρα.
{
    "ID": 3,
    "Title": "Meir Shalev",
    "Body": "Israeli writer (1948–2023)\n\n\nMeir ShalevShalev in 2015Bornמאיר שלו(1948-07-29)29 July 1948Nahalal, IsraelDied11 April 2023(2023-04-11) (aged 74)Alonei Abba, IsraelLanguageHebrewNationalityIsraeliNotable awardsBernstein Prize,Brenner Prize\nMeir Shalev and the theater performance team at the end of the play \"Uncle Aaron and his Rain\" which won the first prize at the Haifa International Children's Theater Festival in 2017\nThe grave of Meir Shalev is covered with flowers after the funeral, April 13, 2023\nMeir Shalev (Hebrew: מאיר שלו; 29 July 1948 – 11 April 2023) was an Israeli writer and newspaper columnist[1] for the daily Yedioth Ahronoth. Shalev's books have been translated into 26 languages.[2]\n\n\nBiography[edit]\nShalev was born in Nahalal, Israel. Later he lived in Jerusalem and at Ginosar with his family. He is the son of the Jerusalem poet Yitzhak Shalev. His cousin Zeruya Shalev is also a w

### Βήμα 2. Προεπεξεργασία κειμένου (Text Processing):

In [13]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

# Κατεβάζουμε τα απαραίτητα δεδομένα για το nltk
nltk.download('punkt')
nltk.download('stopwords')

# Stop words και stemming
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Συνάρτηση για καθαρισμό κειμένου
def preprocess_text(text):
    # Αφαίρεση ειδικών χαρακτήρων
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Μετατροπή σε πεζά γράμματα
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Αφαίρεση stop-words και stemming
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# Προεπεξεργασία όλων των άρθρων
for doc in documents:
    doc['Processed_Body'] = preprocess_text(doc['Body'])

# Αποθήκευση προεπεξεργασμένων δεδομένων
with open('processed_articles.json', 'w', encoding='utf-8') as file:
    json.dump(documents, file, indent=4, ensure_ascii=False)

print("Η προεπεξεργασία ολοκληρώθηκε και τα δεδομένα αποθηκεύτηκαν στο 'processed_articles.json'.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\agram\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\agram\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Η προεπεξεργασία ολοκληρώθηκε και τα δεδομένα αποθηκεύτηκαν στο 'processed_articles.json'.


### Βήμα 3. Ευρετήριο (Indexing):

In [14]:
from collections import defaultdict

# Δημιουργία ανεστραμμένου ευρετηρίου
def create_inverted_index(documents):
    inverted_index = defaultdict(list)  # Λεξικό για το ευρετήριο
    for doc in documents:
        doc_id = doc['ID']
        words = doc['Processed_Body'].split()
        word_counts = defaultdict(int)  # Μετρητής λέξεων στο έγγραφο
        for word in words:
            word_counts[word] += 1
        
        # Ενημέρωση του ανεστραμμένου ευρετηρίου
        for word, count in word_counts.items():
            inverted_index[word].append({'doc_id': doc_id, 'frequency': count})
    
    return inverted_index

# Δημιουργούμε το ευρετήριο
inverted_index = create_inverted_index(documents)

# Αποθήκευση του ευρετηρίου σε αρχείο
with open('inverted_index.json', 'w', encoding='utf-8') as file:
    json.dump(inverted_index, file, indent=4, ensure_ascii=False)

print("Το ανεστραμμένο ευρετήριο δημιουργήθηκε και αποθηκεύτηκε στο 'inverted_index.json'.")


Το ανεστραμμένο ευρετήριο δημιουργήθηκε και αποθηκεύτηκε στο 'inverted_index.json'.


### Βήμα 4. Μηχανή αναζήτησης (Search Engine):
#### α) Επεξεργασία ερωτήματος (Query Processing):

In [21]:
# Συνάρτηση για την επεξεργασία του ερωτήματος
def process_query(query):
    query = re.sub(r'[^a-zA-Z\s]', '', query).lower()
    tokens = word_tokenize(query)
    filtered_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return filtered_tokens

# Συνάρτηση για την εκτέλεση αναζήτησης
def search(query, inverted_index):
    if "AND" in query:
        # Boolean: AND
        terms = query.split("AND")
        if len(terms) == 2:
            term1 = process_query(terms[0].strip())[0]
            term2 = process_query(terms[1].strip())[0]

            docs1 = {doc['doc_id'] for doc in inverted_index.get(term1, [])}
            docs2 = {doc['doc_id'] for doc in inverted_index.get(term2, [])}
            
            results = list(docs1 & docs2)
            return results
        else:
            print("Σφάλμα στο ερώτημα. Χρησιμοποιήστε σωστή σύνταξη: text1 AND text2.")
            return []

    elif "OR" in query:
        # Boolean: OR
        terms = query.split("OR")
        results = set()
        for term in terms:
            term = process_query(term.strip())[0]
            results.update({doc['doc_id'] for doc in inverted_index.get(term, [])})
        return list(results)

    elif "NOT" in query:
        # Boolean: NOT
        term = process_query(query.split("NOT")[1].strip())[0]
        all_docs = {doc['ID'] for doc in documents}
        exclude_docs = {doc['doc_id'] for doc in inverted_index.get(term, [])}
        results = list(all_docs - exclude_docs)
        return results

    else:
        # Μοναδικός όρος αναζήτησης
        term = process_query(query.strip())[0]
        results = [doc['doc_id'] for doc in inverted_index.get(term, [])]
        return results

# Διαρκής αναζήτηση μέχρι ο χρήστης να σταματήσει
while True:
    query = input("Εισάγετε το ερώτημά σας (ή γράψτε 'exit' για τερματισμό): ").strip()
    if query.lower() == 'exit':
        print("Τερματισμός αναζήτησης.")
        break

    results = search(query, inverted_index)

    # Προβολή αποτελεσμάτων
    if results:
        print(f"Βρέθηκαν {len(results)} σχετικά έγγραφα:\n")
        for doc_id in results:
            doc = next((d for d in documents if d['ID'] == doc_id), None)
            print(f"ID: {doc['ID']}, Τίτλος: {doc['Title']}, URL: {doc['URL']}")
    else:
        print("Δε βρέθηκαν σχετικά έγγραφα.")


Εισάγετε το ερώτημά σας (ή γράψτε 'exit' για τερματισμό): exit
Τερματισμός αναζήτησης.


#### β) Κατάταξη αποτελεσμάτων (Ranking):

In [19]:
!pip install scikit-learn rank-bm25

from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import numpy as np

# Προετοιμασία δεδομένων για διαφορετικούς αλγορίθμους
corpus = [doc['Processed_Body'] for doc in documents]

# 1. TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# 2. BM25
bm25 = BM25Okapi([doc.split() for doc in corpus])

# Υπολογισμός κατάταξης με διαφορετικούς αλγορίθμους
def rank_results(query, algorithm="TF-IDF"):
    query_processed = ' '.join(process_query(query))
    
    if algorithm == "TF-IDF":
        # Υπολογισμός σχετικότητας με βάση το TF-IDF
        query_vector = tfidf_vectorizer.transform([query_processed])
        scores = np.dot(tfidf_matrix, query_vector.T).toarray().flatten()
        ranked_indices = np.argsort(-scores)
    
    elif algorithm == "BM25":
        # Υπολογισμός σχετικότητας με βάση το BM25
        scores = bm25.get_scores(query_processed.split())
        ranked_indices = np.argsort(-scores)
    
    elif algorithm == "VSM":
        # Χρήση TF-IDF ως βάση για VSM
        query_vector = tfidf_vectorizer.transform([query_processed])
        scores = np.dot(tfidf_matrix, query_vector.T).toarray().flatten()
        normalized_scores = scores / np.linalg.norm(scores)
        ranked_indices = np.argsort(-normalized_scores)
    
    else:
        raise ValueError("Μη έγκυρος αλγόριθμος. Επιλέξτε 'TF-IDF', 'BM25' ή 'VSM'.")
    
    # Επιστροφή ταξινομημένων αποτελεσμάτων
    return [(documents[idx]['ID'], scores[idx]) for idx in ranked_indices if scores[idx] > 0]

# Διαδραστική αναζήτηση με επιλογή αλγορίθμου
while True:
    query = input("Εισάγετε το ερώτημά σας (ή γράψτε 'exit' για τερματισμό): ").strip()
    if query.lower() == 'exit':
        print("Τερματισμός αναζήτησης.")
        break
    
    algorithm = input("Επιλέξτε αλγόριθμο (TF-IDF, BM25, VSM): ").strip().upper()
    try:
        ranked_results = rank_results(query, algorithm)
        if ranked_results:
            print(f"Βρέθηκαν {len(ranked_results)} σχετικά έγγραφα με τον αλγόριθμο {algorithm}:\n")
            for rank, (doc_id, score) in enumerate(ranked_results[:10], start=1):
                doc = next((d for d in documents if d['ID'] == doc_id), None)
                print(f"{rank}. ID: {doc['ID']}, Τίτλος: {doc['Title']}, Σκορ: {score:.4f}, URL: {doc['URL']}")
        else:
            print("Δε βρέθηκαν σχετικά έγγραφα.")
    except ValueError as e:
        print(e)


Εισάγετε το ερώτημά σας (ή γράψτε 'exit' για τερματισμό): open
Επιλέξτε αλγόριθμο (TF-IDF, BM25, VSM): TF-IDF
Βρέθηκαν 175 σχετικά έγγραφα με τον αλγόριθμο TF-IDF:

1. ID: 138, Τίτλος: Andrew Campbell (golfer), Σκορ: 0.2732, URL: https://en.wikipedia.org/wiki/Andrew_Campbell_(golfer)
2. ID: 281, Τίτλος: 2024 Córdoba Open, Σκορ: 0.1364, URL: https://en.wikipedia.org/wiki/2024_C%C3%B3rdoba_Open
3. ID: 369, Τίτλος: Pakistan Open, Σκορ: 0.1085, URL: https://en.wikipedia.org/wiki/Pakistan_Open
4. ID: 656, Τίτλος: Piano Trio No. 43 (Haydn), Σκορ: 0.0747, URL: https://en.wikipedia.org/wiki/Piano_Trio_No._43_(Haydn)
5. ID: 607, Τίτλος: 2004 Wimbledon Championships – Mixed doubles, Σκορ: 0.0733, URL: https://en.wikipedia.org/wiki/2004_Wimbledon_Championships_%E2%80%93_Mixed_doubles
6. ID: 661, Τίτλος: Costa Rica at the 2013 World Aquatics Championships, Σκορ: 0.0635, URL: https://en.wikipedia.org/wiki/Costa_Rica_at_the_2013_World_Aquatics_Championships
7. ID: 565, Τίτλος: 2011 Australian Open –

### Βήμα 5. Αξιολόγηση συστήματος: 

In [20]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Παράδειγμα δοκιμαστικών ερωτημάτων και αναμενόμενων απαντήσεων
test_queries = [
    {"query": "Israeli writer", "relevant_docs": [3]},
    {"query": "animal actors", "relevant_docs": [1]},
    {"query": "US Army general", "relevant_docs": [10]},
    {"query": "Punjabi culture", "relevant_docs": [4]},
]

# Συνάρτηση για αξιολόγηση
def evaluate_search_engine(algorithm="TF-IDF"):
    precisions = []
    recalls = []
    f1_scores = []
    average_precisions = []
    
    for test in test_queries:
        query = test['query']
        relevant_docs = set(test['relevant_docs'])

        # Λήψη αποτελεσμάτων με κατάταξη
        ranked_results = rank_results(query, algorithm)
        retrieved_docs = set(doc_id for doc_id, _ in ranked_results)

        # Υπολογισμός Precision, Recall, F1
        tp = len(retrieved_docs & relevant_docs)  # True Positives
        fp = len(retrieved_docs - relevant_docs)  # False Positives
        fn = len(relevant_docs - retrieved_docs)  # False Negatives

        precision = tp / (tp + fp) if tp + fp > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        # Υπολογισμός Average Precision (AP)
        average_precision = 0
        for k, (doc_id, _) in enumerate(ranked_results):
            if doc_id in relevant_docs:
                precision_at_k = len(relevant_docs & set([doc_id for doc_id, _ in ranked_results[:k+1]])) / (k + 1)
                average_precision += precision_at_k
        average_precision /= len(relevant_docs) if relevant_docs else 1
        average_precisions.append(average_precision)

    # Υπολογισμός μέσων τιμών
    mean_precision = sum(precisions) / len(precisions)
    mean_recall = sum(recalls) / len(recalls)
    mean_f1 = sum(f1_scores) / len(f1_scores)
    mean_ap = sum(average_precisions) / len(average_precisions)

    print(f"Αποτελέσματα για τον αλγόριθμο {algorithm}:")
    print(f"- Μέση Ακρίβεια (Precision): {mean_precision:.4f}")
    print(f"- Μέση Ανάκληση (Recall): {mean_recall:.4f}")
    print(f"- Μέσο F1-Score: {mean_f1:.4f}")
    print(f"- Μέση Ακρίβεια (MAP): {mean_ap:.4f}")

# Εκτέλεση αξιολόγησης για όλους τους αλγορίθμους
for algo in ["TF-IDF", "BM25", "VSM"]:
    evaluate_search_engine(algo)


Αποτελέσματα για τον αλγόριθμο TF-IDF:
- Μέση Ακρίβεια (Precision): 0.0120
- Μέση Ανάκληση (Recall): 1.0000
- Μέσο F1-Score: 0.0236
- Μέση Ακρίβεια (MAP): 0.4811
Αποτελέσματα για τον αλγόριθμο BM25:
- Μέση Ακρίβεια (Precision): 0.0120
- Μέση Ανάκληση (Recall): 1.0000
- Μέσο F1-Score: 0.0236
- Μέση Ακρίβεια (MAP): 0.7500
Αποτελέσματα για τον αλγόριθμο VSM:
- Μέση Ακρίβεια (Precision): 0.0120
- Μέση Ανάκληση (Recall): 1.0000
- Μέσο F1-Score: 0.0236
- Μέση Ακρίβεια (MAP): 0.4811
